In [1]:
import numpy as np
import pandas as pd
import RIfunctions as ri

In [2]:
application_df = pd.read_csv('input_file_1.csv', sep = ',', index_col=0)
accept_df = pd.read_csv('input_file_2.csv', sep = ',', index_col=0)

C:\Users\joshy\AppData\Roaming\Python\Python38\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
# Filter for instances that were before 2016 as we know the accepted loans default status
# and should only infer those rejected applications for the same period

accept_df['issue_d'] = pd.to_datetime(accept_df['issue_d'])
application_df['issue_d'] = pd.to_datetime(application_df['issue_d'])

accept_df = accept_df.loc[accept_df['issue_d'] < '2016-1-1']
application_df = application_df.loc[application_df['issue_d'] < '2016-1-1']

In [4]:
application_df['rejected'].value_counts()

1    5704123
0     646878
Name: rejected, dtype: int64

In [5]:
accept_df['charged_off'].value_counts()

0    516026
1    130852
Name: charged_off, dtype: int64

In [6]:
accept_df.shape

(646878, 21)

In [7]:
rejects = application_df.loc[application_df["rejected"] ==1]

In [8]:
# Simple assignment

ri_df = pd.concat([accept_df, rejects], axis=0, ignore_index=True)
ri_df.loc[ri_df['rejected'] == 1, 'charged_off'] = 1


In [9]:
# Random assignment

# neg, pos = np.bincount(accept_df['charged_off'])
# total = neg + pos
# reject_rate = pos / total

# print('Default Rate:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
#     total, pos, 100 * pos / total))

# random_assign_rate = reject_rate * 3.5

# print(reject_rate)
# print(random_assign_rate)

# # Higher the fraction, bigger the sample
# split_BAD = rejects.sample(frac = random_assign_rate, random_state=1)
# split_GOOD = rejects.drop(split_BAD.index)

# split_BAD['charged_off'] = 1
# split_GOOD = split_GOOD.assign(charged_off= 0)

# ri_df = pd.concat([accept_df, split_BAD, split_GOOD], axis=0, ignore_index=True)

In [10]:
ri_df['charged_off'].value_counts()

1.0    5834975
0.0     516026
Name: charged_off, dtype: int64

In [11]:
print(application_df.shape)
print(ri_df.shape)

(6351001, 6)
(6351001, 22)


In [12]:
ri_df.drop('rejected', axis=1, inplace=True)

In [13]:
nan_cols = [i for i in ri_df.columns if ri_df[i].isna().any()]
nan_cols = [e for e in nan_cols if e not in ('emp_length', 'purpose')]

In [14]:
ri_df.drop(nan_cols, axis=1, inplace=True)

In [15]:
ri_df = ri.purposeCleaning(ri_df)

In [16]:
print(ri_df.shape)

(6149437, 6)


In [17]:
ri_df.to_csv('RI_Accept.csv', mode='w+')